In [ ]:
import ROOT as root
import root_pandas as rp
import uproot
import sys
import os
import analysis_variables as a_v
import numpy as np
import pandas as pd
from pylab import *
from array import array
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import probfit
import iminuit
import importlib
import statistics
from uncertainties import unumpy
import mplhep as hep
import hist as hist_lib
import tensorflow as tf
import zfit
from zfit import z
import zfit.z.numpy as znp
import warnings
from tqdm.notebook import tqdm
importlib.reload(a_v)

parameters_created = {}
parameters_created['e'] = 0
parameters_created['mu'] = 0
parameters_created['pi'] = 0
parameters_created['rho'] = 0

skip_br = 0

yield_bkg = {}
coeff_br = {}
yield_sig = {}

In [ ]:
importlib.reload(a_v)

mc_type = 'ri'
long = 1

inFile = {}

if mc_type == 'ri':
    for event in a_v.event_type:
        if long == 1:
            print('Reading from', a_v.inFilePath_long[event])
        else:
            print('Reading from', a_v.inFilePath[event])

In [ ]:
importlib.reload(a_v)

list_variables = ['...','...'] 

lumi = 'data'
multiplicity_var = '...'
ROE_var = '...'
ROE_blind_cut = 0.5

if mc_type == 'wc':
    ROE_blind_cut = -1

var_contReweight = 'contReweight'

in_cut = {}
in_cut['e'] = '...>0.01 and ...>0.5 and ...<0.8 and ...<1'
in_cut['mu'] = '...>0.01 and ...>0.5 and ...<0.8 and ...<1'
in_cut['pi'] = '...>0.01 and ...>1.4 and ...<0.6 and ...<1'
in_cut['rho'] = '...>0.01 and ...>1.65 and ...<0.7 and ...<1


cut_pdg = '... != 2 and ... != 20 and ... != 2 and ... != 8'

cut_sig_pdg = {}
cut_sig_pdg['e'] = '... == 1 or ... == 1'
cut_sig_pdg['mu'] = '... == 1 or ... == 1'
cut_sig_pdg['pi'] = '... == 1 or ... == 1'
cut_sig_pdg['rho'] = '... == 1 or ... == 1'
cut_sig_pdg['other'] = 'not(' + cut_sig_pdg['e'] + ') and not(' + cut_sig_pdg['mu'] + ') and not(' + cut_sig_pdg['pi'] + ') and not(' + cut_sig_pdg['rho'] + ')'

base_cut = ' and ...>5.27 and ...>-0.15 and ...<0.1 and ...<0.6'

cuts = {}
cuts_extra = {}
cuts_str = {}
for event in a_v.event_type:
    cuts[event] = {}
    cuts_extra[event] = {}
    for mode in a_v.decay_type:
        if not in_cut[mode]:
            cuts_str[mode] = a_v.decay_cuts_pandas[mode] + base_cut
            cuts[event][mode] = inFile[event].query(cuts_str[mode])
        else:
            if event == '...':
                cuts_str[mode] =a_v.decay_cuts_pandas[mode] + ' and ' + in_cut[mode] + base_cut + ' and ' + cut_pdg
            else:
                cuts_str[mode] =a_v.decay_cuts_pandas[mode] + ' and ' + in_cut[mode] + base_cut
            cuts[event][mode] = inFile[event].query(cuts_str[mode])

In [ ]:
dirName = {}
for var in [ROE_var]:
    dirName[var] = {}
    for mode in a_v.decay_type:
        dirName[var][mode] = a_v.output_dir + var
        try:
            os.mkdir(dirName[var][mode])
            print("Directory " , dirName[var][mode] ,  " Created") 
        except FileExistsError:
            print("Directory " , dirName[var][mode] ,  " already exists")
        try:
            os.mkdir(dirName[var][mode] + '/fit')
            print("Directory " , dirName[var][mode] +'/fit' ,  " Created") 
        except FileExistsError:
            print("Directory " , dirName[var][mode] +'/fit' ,  " already exists")

In [ ]:
importlib.reload(a_v)
warnings.simplefilter('ignore')

plt.rcParams.update({
          'font.size': 20,
          'figure.figsize': (11, 10),
          'axes.grid': False,
          'grid.linestyle': '-',
          'grid.alpha': 0.2,
          'lines.markersize': 5.0,
          'xtick.minor.visible': True,
          'xtick.direction': 'in',
          'xtick.major.size': 10.0,
          'xtick.minor.size': 5.0,
          'xtick.top': True,
          'ytick.minor.visible': True,
          'ytick.direction': 'in',
          'ytick.major.size': 10.0,
          'ytick.minor.size': 5.0,
          'ytick.right': True,
          'errorbar.capsize': 0.0,
          'figure.max_open_warning':100
        })

sig_scale = 500

bool_sig_scale = 0 #0 or 1
sig_on_top = 1
log_scale = 0 
scale_to_offres = 1 
scale_fit = 1 
scale_bb_to_data = 0
show_chi_2 = 1 
no_data = 1 
subtract_cont = 0 
shift_fit = 0 
multiplicity_corr = 1 
print_ratio = 1 

start_bin_ = 0

scale_cont = {}
scale_bbar = {}
scale_sig = {}

fit_par_1 = {}
fit_error_1 = {}
fit_par_2 = {}
fit_error_2 = {}
corr_matrix = {}
frame = {}
gr = {}

scale_fit_bbar = {}
scale_fit_bbar['e'] = a_v.scale_fit_bbar['e']
scale_fit_bbar['mu'] = a_v.scale_fit_bbar['mu']
scale_fit_bbar['pi'] = a_v.scale_fit_bbar['pi']
scale_fit_bbar['rho'] = a_v.scale_fit_bbar['rho']

offres_scale = {}
offres_scale['e'] = a_v.offres_scale['e']
offres_scale['mu'] = a_v.offres_scale['mu']
offres_scale['pi'] = a_v.offres_scale['pi']
offres_scale['rho'] = a_v.offres_scale['rho'] 

#import seaborn as sns
#plt.style.use('belle2')
#sns.set()
#sns.set_theme(style="white", palette='colorblind')

if no_data == 1 or mbc_sideband == 1:
    ROE_blind_cut = -1

for var in list_variables:
    if var == ROE_var:
        start_bin = 4
    else:
        start_bin = start_bin_
    if shift_fit == 1:
        start_bin = 4
    scale_bbar[var] = {}
    scale_cont[var] = {}
    scale_sig[var] = {}
    if mc_type != 'wc':
        if var != ROE_var:
            for i in a_v.event_type:
                if i == 'sig_scale':
                    continue
                for j in a_v.decay_type:
                    cuts[i][j][var] = cuts[i][j].query(f'{ROE_var}>{ROE_blind_cut}')[var]
        if var == ROE_var:
            for j in a_v.decay_type:
                pd.options.mode.chained_assignment = None
                cuts['data'][j][var] = cuts['data'][j].query(f'{ROE_var}>{ROE_blind_cut}')[var]

    for mode in a_v.decay_type:
        if 'd1_d0_d1' in var or 'd1_M' in var:
            if mode != 'rho':
                continue
        if lumi == 'data':
            if no_data == 0:
                fig, ax = plt.subplots(nrows=2, 
                                       ncols=1, 
                                       gridspec_kw={'height_ratios': [3, 1]},
                                       sharex=True, 
                                      )
                plt.subplots_adjust(hspace=0.05)
            else:
                fig, ax = plt.subplots()


        if lumi == 'data' and mc_type == 'ri':
            scale = {}
            scale_tot = []
            for i in a_v.event_type:
                if i != 'data':
                    if i != 'offres':
                        if i == 'ccbar' or i == 'ddbar' or i == 'ssbar' or i == 'uubar' or i =='taupair':
                            if mode == 'mu':
                                scale[i] = a_v.scale_data_lumi[i]*(cuts[i][mode])[var_contReweight].to_numpy() #+f'_{mode}'
                            else:
                                scale[i] = a_v.scale_data_lumi[i]*(cuts[i][mode])[var_contReweight].to_numpy()
                        else:
                            cuts[i][mode]['...'] = np.ones(len(cuts[i][mode]))        
                            if mode == 'e':
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 211, cuts[i][mode]['...'], cuts[i][mode]['...']) 
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 321, cuts[i][mode]['...'], cuts[i][mode]['...'])
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 11, cuts[i][mode]['...'], cuts[i][mode]['...'])
                            if mode == 'mu':
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 211, cuts[i][mode]['...'], cuts[i][mode]['...']) 
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 321, cuts[i][mode]['...'], cuts[i][mode]['...'])
                                cuts[i][mode]['...'] = np.where(abs(cuts[i][mode]['...']) == 13, cuts[i][mode]['...'], cuts[i][mode]['...'])
                            if mode == 'pi' or mode == 'rho':
                                cuts[i][mode]['...'] = cuts[i][mode]['...']

                            cuts[i][mode]['...'] = cuts[i][mode]['...'].replace(np.nan, 0)
                            cuts[i][mode]['...'] = np.where(cuts[i][mode]['...'] > 3, 0, cuts[i][mode]['...'])
                            
                            scale[i] = a_v.scale_data_lumi[i]*cuts[i][mode].PIDWeight.to_numpy()
                            
                    elif i == '...':
                        scale[i] = np.array([a_v.scale_data_lumi[i]]*len((cuts[i][mode])[var]))
                    if i == '...' and scale_fit == 1:
                        scale[i] = scale[i] * cuts[i][mode].column
                    if i == '...' and scale_fit == 1:
                        scale[i] = scale[i] * cuts[i][mode].column
                    if i == '...' and scale_fit == 1:
                        scale[i] = scale[i] * cuts[i][mode].column
                    if i == '...' and multiplicity_corr == 1:
                        scale[i] = np.prod(np.array([scale[i],np.array((a_v.inter[mode] + ((cuts[i][mode])[multiplicity_var]+0.5) * a_v.coeff[mode]))]),axis=0)
                    if i == '...' and multiplicity_corr == 1:
                        scale[i] = np.prod(np.array([scale[i],np.array((a_v.inter[mode] + ((cuts[i][mode])[multiplicity_var]+0.5) * a_v.coeff[mode]))]),axis=0)
                    if i == '...' and multiplicity_corr == 1:
                        scale[i] = np.prod(np.array([scale[i],np.array((a_v.inter[mode] + ((cuts[i][mode])[multiplicity_var]+0.5) * a_v.coeff[mode]))]),axis=0)
                        
        continuum = []
        continuum = pd.concat([(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var]])
        if scale_to_offres == 1:
            for i in ['...','...','...','...','...']:
                scale[i] = scale[i] * offres_scale[mode]

        scale['cont'] = np.concatenate([scale['...'],scale['...'],scale['...'],scale['...'],scale['...']])
        scale['sig_scale'] = sig_scale * scale['sig']
        
        scale_bbar[var][mode] = np.concatenate([scale['...'],scale['...']])
        scale_cont[var][mode] = scale['...']
        scale_sig[var][mode] = scale['...']

        tot = []
        hist_list = []
        if sig_on_top == 0:
            hist_list.append((cuts['sig'][mode])[var])
            hist_list.append((continuum))
            hist_list.append((cuts['mixed'][mode][var]))
            hist_list.append((cuts['charged'][mode][var]))
            tot = pd.concat(hist_list)
            scale_tot = np.concatenate([scale['sig'],scale['cont'],scale['mixed'],scale['charged']])
        else:
            hist_list.append((continuum))
            hist_list.append((cuts['mixed'][mode][var]))
            hist_list.append((cuts['charged'][mode][var]))
            hist_list.append((cuts['sig'][mode])[var])
            tot = pd.concat(hist_list)
            scale_tot = np.concatenate([scale['cont'],scale['mixed'],scale['charged'],scale['sig']])
            
        if lumi == 'data':
            if var != '...':
                data_hist, data_bins = np.histogram((cuts['data'][mode])[var].to_numpy(), 
                                                        range=(a_v.variables_par[f'{var}']['range_low'], 
                                                               a_v.variables_par[f'{var}']['range_high']),
                                                        bins=a_v.variables_par[f'{var}']['bins']
                                                       )
            else:
                n_edges = a_v.variables_par[f'{var}']['bins'] + 1
                edges = np.logspace(-3, 0, n_edges)
                data_hist, data_bins = np.histogram((cuts['data'][mode])[var].to_numpy(), 
                                                    range=(a_v.variables_par[f'{var}']['range_low'], 
                                                           a_v.variables_par[f'{var}']['range_high']),
                                                    bins=edges
                                                   )

            data_hist_unp = unumpy.uarray(data_hist,np.sqrt(data_hist))
            data_hist = unumpy.nominal_values(data_hist_unp)
            data_hist_errors = unumpy.std_devs(data_hist_unp)
            data_bin_center = (data_bins[1:]+data_bins[:-1])/2

            hist_err = {}
            for j in ['...','...','...','...','...','...','...','...']:
                if var != '...':
                    hist_err[j], bin_err = np.histogram((cuts[j][mode])[var].to_numpy(), 
                                                        range=(a_v.variables_par[f'{var}']['range_low'], 
                                                               a_v.variables_par[f'{var}']['range_high']),
                                                        bins=a_v.variables_par[f'{var}']['bins'],
                                                        weights=scale[j]
                                                       )
                else:
                    n_edges = a_v.variables_par[f'{var}']['bins'] + 1
                    edges = np.logspace(-3, 0, n_edges)
                    hist_err[j], bin_err = np.histogram((cuts[j][mode])[var].to_numpy(), 
                                                        range=(a_v.variables_par[f'{var}']['range_low'], 
                                                               a_v.variables_par[f'{var}']['range_high']),
                                                        bins=edges,
                                                        weights=scale[j]
                                                       )


            tot_hist_unp = np.zeros(int(a_v.variables_par[f'{var}']['bins']))
            cont_hist_unp = np.zeros(int(a_v.variables_par[f'{var}']['bins']))
            mc_unp = {}
            for j in ['...','...','...','...','...','...','...','...']: 
                if np.sum(hist_err[j])==0:
                    mc_unp[j] = unumpy.uarray(hist_err[j],np.sqrt(hist_err[j]))
                else:
                    mc_unp[j] = unumpy.uarray(hist_err[j],np.sqrt(hist_err[j])*np.mean(scale[j]))
                tot_hist_unp = tot_hist_unp + mc_unp[j]
                if j in ['...','...','...','...','...']:
                    cont_hist_unp = cont_hist_unp + mc_unp[j]

            tot_hist = unumpy.nominal_values(tot_hist_unp)
            tot_hist_errors = unumpy.std_devs(tot_hist_unp)
            cont_hist = unumpy.nominal_values(cont_hist_unp)
            
            data_no_cont_unp = data_hist_unp - cont_hist_unp
            data_no_cont = unumpy.nominal_values(data_no_cont_unp)
            data_no_cont_errors = unumpy.std_devs(data_no_cont_unp)
            
            tot_no_cont_unp = tot_hist_unp - cont_hist_unp
            tot_no_cont = unumpy.nominal_values(tot_no_cont_unp)
            tot_no_cont_errors = unumpy.std_devs(tot_no_cont_unp)
            
            data_mc_no_cont_ratio = np.divide(data_no_cont,tot_no_cont,out=np.zeros_like(tot_no_cont),where=(tot_no_cont > 0))
            data_mc_no_cont_ratio_errors = data_mc_no_cont_ratio * np.sqrt(np.divide(data_no_cont_errors,data_no_cont,where=(data_no_cont > 0))**2 + np.divide(tot_no_cont_errors,tot_no_cont,where=(tot_no_cont > 0))**2)
            
            if start_bin != 0:
                ratio_all = np.sum(data_hist[start_bin:])/np.sum(tot_hist[start_bin:])
                ratio_no_cont = (np.sum(data_hist[start_bin:])-np.sum(cont_hist[start_bin:]))/(np.sum(tot_hist[start_bin:])-np.sum(cont_hist[start_bin:]))
                print('Data/MC ratio:', np.sum(data_hist[start_bin:])/np.sum(tot_hist[start_bin:]))
                print('Data-cont/bb ratio:', (np.sum(data_hist[start_bin:])-np.sum(cont_hist[start_bin:]))/(np.sum(tot_hist[start_bin:])-np.sum(cont_hist[start_bin:])))
            else:
                ratio_all = np.sum(data_hist)/np.sum(tot_hist)
                ratio_no_cont = (np.sum(data_hist)-np.sum(cont_hist))/(np.sum(tot_hist)-np.sum(cont_hist))
                print('Data/MC ratio:', np.sum(data_hist)/np.sum(tot_hist))
                print('Data-cont/bb ratio:', (np.sum(data_hist)-np.sum(cont_hist))/(np.sum(tot_hist)-np.sum(cont_hist)))
                
        if scale_bb_to_data == 1:
            for i in ['...','...','...']:
                scale[i] = scale[i] * ratio_no_cont

        hist_color = []
        hist_label = []
        hist_scale = []
        if sig_on_top == 0:
            for j in ['...','...','...','...']:
                hist_color.append(a_v.hist_color_plt[j])
                hist_label.append(a_v.event_type_ext_pandas[j])
                hist_scale.append(scale[j])
        else:
            for j in ['...','...','...','...']:
                hist_color.append(a_v.hist_color_plt[j])
                hist_label.append(a_v.event_type_ext_pandas[j])
                hist_scale.append(scale[j])
                
        if subtract_cont == 1:
            hist_list.remove(continuum) 
            hist_color.remove(a_v.hist_color_plt['cont'])
            hist_label.remove(a_v.event_type_ext_pandas['cont'])
            hist_scale.remove(scale['cont'])
            data_hist = data_no_cont
            data_hist_errors = data_no_cont_errors
            err_list = ['...','...','...']
        else:
            err_list = ['...','...','...','...','...','...','...','...']

        if no_data == 0:
            if var != '...':
                hist = ax[0].hist(hist_list,
                                  bins=a_v.variables_par[f'{var}']['bins'], 
                                  range=(a_v.variables_par[f'{var}']['range_low'], 
                                         a_v.variables_par[f'{var}']['range_high']),
                                  histtype='stepfilled',
                                  edgecolor='black',
                                  linewidth=2,
                                  stacked=True,
                                  color=hist_color,
                                  label=hist_label,
                                  weights=hist_scale
                                 )
            else:
                n_edges = a_v.variables_par[f'{var}']['bins'] + 1
                edges = np.logspace(-3, 0, n_edges)
                hist = ax[0].hist(hist_list,
                                  bins=edges, 
                                  range=(a_v.variables_par[f'{var}']['range_low'], 
                                         a_v.variables_par[f'{var}']['range_high']),
                                  histtype='stepfilled',
                                  edgecolor='black',
                                  linewidth=2,
                                  stacked=True,
                                  color=hist_color,
                                  label=hist_label,
                                  weights=hist_scale
                                 )
        else:
            if var != '...':
                hist = ax.hist(hist_list,
                                  bins=a_v.variables_par[f'{var}']['bins'], 
                                  range=(a_v.variables_par[f'{var}']['range_low'], 
                                         a_v.variables_par[f'{var}']['range_high']),
                                  histtype='stepfilled',
                                  edgecolor='black',
                                  linewidth=2,
                                  stacked=True,
                                  color=hist_color,
                                  label=hist_label,
                                  weights=hist_scale
                                 )
            else:
                n_edges = a_v.variables_par[f'{var}']['bins'] + 1
                edges = np.logspace(-3, 0, n_edges)
                hist = ax.hist(hist_list,
                                  bins=edges, 
                                  range=(a_v.variables_par[f'{var}']['range_low'], 
                                         a_v.variables_par[f'{var}']['range_high']),
                                  histtype='stepfilled',
                                  edgecolor='black',
                                  linewidth=2,
                                  stacked=True,
                                  color=hist_color,
                                  label=hist_label,
                                  weights=hist_scale
                                 )

        if log_scale != 1 and bool_sig_scale == 1:
            if no_data ==0:
                hist = ax[0].hist((cuts['sig'][mode])[var],
                                  bins=a_v.variables_par[f'{var}']['bins'], 
                                  range=(a_v.variables_par[f'{var}']['range_low'], 
                                         a_v.variables_par[f'{var}']['range_high']),
                                  histtype='step',
                                  edgecolor=a_v.hist_color_plt['sig'],
                                  linewidth=3,
                                  label=f'Sig x{sig_scale}',
                                  weights=scale['sig_scale']
                                 )
            else:
                hist = ax.hist((cuts['sig_scale'][mode])[var],
                              bins=a_v.variables_par[f'{var}']['bins'], 
                              range=(a_v.variables_par[f'{var}']['range_low'], 
                                     a_v.variables_par[f'{var}']['range_high']),
                              histtype='step',
                              edgecolor=a_v.hist_color_plt['sig'],
                              linewidth=3,
                              label=f'Sig x{sig_scale}',
                              weights=scale['sig_scale']
                             )       
                
        if no_data == 0:
            if var != '...':
                hist = ax[0].errorbar(x=data_bin_center,
                                      y=data_hist, 
                                      xerr=np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center)),
                                      yerr=data_hist_errors,
                                      fmt='ko',
                                      label=a_v.event_type_ext_pandas['data']
                                     )
            else:
                hist = ax[0].errorbar(x=data_bin_center,
                                      y=data_hist, 
                                      yerr=data_hist_errors,
                                      fmt='ko',
                                      label=a_v.event_type_ext_pandas['data']
                                     )

            
            
        hist_err = {}
        for j in ['...','...','...','...','...','...','...','...','...']:
            if var != '...':
                hist_err[j], bin_err = np.histogram((cuts[j][mode])[var].to_numpy(), 
                                                    range=(a_v.variables_par[f'{var}']['range_low'], 
                                                           a_v.variables_par[f'{var}']['range_high']),
                                                    bins=a_v.variables_par[f'{var}']['bins'],
                                                    weights=scale[j]
                                                   )
            else:
                hist_err[j], bin_err = np.histogram((cuts[j][mode])[var].to_numpy(), 
                                                    range=(a_v.variables_par[f'{var}']['range_low'], 
                                                           a_v.variables_par[f'{var}']['range_high']),
                                                    bins=edges,
                                                    weights=scale[j]
                                                   )


        tot_hist_unp = np.zeros(int(a_v.variables_par[f'{var}']['bins']))
        cont_hist_unp = np.zeros(int(a_v.variables_par[f'{var}']['bins']))
        mc_unp = {}
        for j in err_list: 
            if np.sum(hist_err[j])==0:
                mc_unp[j] = unumpy.uarray(hist_err[j],np.sqrt(hist_err[j]))
            else:
                mc_unp[j] = unumpy.uarray(hist_err[j],np.sqrt(hist_err[j])*np.mean(scale[j]))
            tot_hist_unp = tot_hist_unp + mc_unp[j]
            if j in ['...','...','...','...','...']:
                cont_hist_unp = cont_hist_unp + mc_unp[j]

        tot_hist = unumpy.nominal_values(tot_hist_unp)
        tot_hist_errors = unumpy.std_devs(tot_hist_unp)


        if var == '...':
            data_bin_center = []
            for i in range(len(edges)-1):
                data_bin_center.append((edges[i]+edges[i+1])/2)

        for i in range(len(data_bin_center)):
            if i == len(data_bin_center)-1:    
                rect = plt.Rectangle((bin_err[i], tot_hist[i] - tot_hist_errors[i]), bin_err[i+1]-bin_err[i], 2 * tot_hist_errors[i],
                                 facecolor='salmon', alpha=0.2, hatch='///', edgecolor='darkred', label = 'MC Unc')
            else:   
                rect = plt.Rectangle((bin_err[i], tot_hist[i] - tot_hist_errors[i]), bin_err[i+1]-bin_err[i], 2 * tot_hist_errors[i],
                                 facecolor='salmon', alpha=0.2, hatch='///', edgecolor='darkred')

            if no_data == 0:
                ax[0].add_patch(rect)
            else:
                ax.add_patch(rect)
                    
        if no_data == 0:

            data_mc_ratio = np.divide(data_hist,tot_hist,out=np.zeros_like(tot_hist),where=(tot_hist > 0))
            data_mc_ratio_errors = data_mc_ratio * np.sqrt(np.divide(data_hist_errors,data_hist,where=(data_hist > 0))**2 + np.divide(tot_hist_errors,tot_hist,where=(tot_hist > 0))**2)
            
            if var != '...':
                hist = ax[1].errorbar(x=data_bin_center,
                                      y=data_mc_ratio,
                                      xerr=np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center)),
                                      yerr=data_mc_ratio_errors,
                                      fmt='ko'
                                     )
            else:
                hist = ax[1].errorbar(x=data_bin_center,
                                      y=data_mc_ratio,
                                      yerr=data_mc_ratio_errors,
                                      fmt='ko'
                                     )  

            if show_chi_2 == 1:
                if start_bin != 0:
                    g = root.TGraphErrors(len(data_bin_center[start_bin:]),
                                          data_bin_center[start_bin:],
                                          data_mc_ratio[start_bin:],
                                          np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center))[start_bin:],
                                          data_mc_ratio_errors[start_bin:]
                                         )
                else:
                    g = root.TGraphErrors(len(data_bin_center[:]),
                                          data_bin_center[:],
                                          data_mc_ratio[:],
                                          np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center))[:],
                                          data_mc_ratio_errors[:]
                                         )

                f = root.TF1("f","[0]")
                f.SetParNames("P1")
                f.SetParameter(0,1)

                result = g.Fit(f,"S,Q")
                ndf = f.GetNDF()
                chi2 = result.Chi2()
                print('chi2:',chi2, 'ndf:',ndf)

        if lumi == 'data':
            if no_data == 0:
                ax[0].legend(ncol=4, fontsize='small', frameon=False, loc='upper right', columnspacing=0.5)
                ax[1].set_xlabel(a_v.variables_par[f'{var}']['x_axis'], fontsize=26)
                ax[0].set_ylabel(a_v.variables_par[f'{var}']['y_axis'], fontsize=26)
                ax[1].set_ylabel('Data/MC',fontsize=26)
                ax[0].set_xlim(a_v.variables_par[f'{var}']['range_low'], 
                            a_v.variables_par[f'{var}']['range_high'])
                if log_scale == 1:
                    ax[0].set_yscale('log')
                    ax[0].set_ylim(0.1,10000)
                    ax[0].set_xscale('log')
                    ax[0].set_xlim(0,1)
                current_ylim = ax[0].get_ylim()
                y_range = current_ylim[1] 
                new_ylim = (0, 1.25 * y_range)
                ax[0].set_ylim(new_ylim)
                ax[1].set_ylim(0.5,1.5)
                ax[1].hlines(1,
                            a_v.variables_par[f'{var}']['range_low'], 
                            a_v.variables_par[f'{var}']['range_high'],
                            color='black'
                            )
                high = a_v.variables_par[f'{var}']['range_high']
                low = a_v.variables_par[f'{var}']['range_low']
                ax[0].text(0.01 * high + 0.99 * low, new_ylim[1] * 1.02, r'Belle II preliminary:', fontsize = 20, style='italic')
                ax[0].text(0.02 * high + 0.99 * low, new_ylim[1] * 0.92, r'$\int \mathcal{L} dt = \,fb^{-1}$', fontsize = 20, style='italic')
                ax[0].text(0.02 * high + 0.99 * low, new_ylim[1] * 0.85, a_v.latex_decay[mode], fontsize = 20, style='italic')
                
                if show_chi_2 == 1:
                    ax[1].text(a_v.variables_par[f'{var}']['range_low'], 1.55, r'$\chi^2$/ndf='+f'{round(chi2,1)}/{ndf}', fontsize = 24,style='italic')
                if print_ratio == 1:
                    ax[0].set_xlabel(f'Data/MC Ratio={round(ratio_all,3)}', fontsize = 22, style='italic')
                    ax[1].text(0.7*a_v.variables_par[f'{var}']['range_high']+0.3*a_v.variables_par[f'{var}']['range_low'], 1.55, r'$B \overline{B}$ ' + f'Ratio={round(ratio_no_cont,3)}', fontsize = 24, style='italic')
                
                if shift_fit == 1:
                    
                    last_bin = 13
                    
                    if start_bin != 0:
                        g = root.TGraphErrors(len(data_bin_center[start_bin:last_bin]),
                                              data_bin_center[start_bin:last_bin],
                                              data_mc_ratio[start_bin:last_bin],
                                              np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center))[start_bin:last_bin],
                                              data_mc_ratio_errors[start_bin:last_bin]
                                             )
                    else:
                        g = root.TGraphErrors(len(data_bin_center[:last_bin]),
                                              data_bin_center[:last_bin],
                                              data_mc_ratio[:last_bin],
                                              np.array([((a_v.variables_par[f'{var}']['range_high']-a_v.variables_par[f'{var}']['range_low'])/a_v.variables_par[f'{var}']['bins']*0.5)]*len(data_bin_center))[:last_bin],
                                              data_mc_ratio_errors[:last_bin]
                                             )
                    

                    f = root.TF1("f","[0]*x+[1]")
                    f.SetParNames("P1","P2")
                    f.SetParameter(1,1)
                    f.SetParameter(0,0)

                    result = g.Fit(f,"S")
                    
                    coef = f.GetParameter(0)
                    inter = f.GetParameter(1)

                    chi2 = result.Chi2()
                    ndf = f.GetNDF()
                    
                    fit_par_1[mode] = round(coef,6)
                    fit_error_1[mode] = round(f.GetParError(0),6)
                    fit_par_2[mode] = round(inter,6)
                    fit_error_2[mode] = round(f.GetParError(1),6)
                    corr_matrix[mode] = result.GetCorrelationMatrix()
                    corr_matrix[mode].Print()
                    print(mode, 'coef = ', fit_par_1[mode], '+/-', fit_error_2[mode], 'inter =', fit_par_2[mode], '+/-', fit_error_2[mode])
                    
                    if print_contour == 1:
                        gr[mode] = root.TGraph(100)
                        frame[mode] = result.Contour(0, 1, gr[mode])

                    factor = (coef*data_bin_center+inter)
                    ax[1].plot(data_bin_center,factor,
                              color='red'
                              )
            else:
                ax.legend(ncol=3, fontsize='small', frameon=False, loc='upper right', columnspacing=1)
                ax.set_xlabel(a_v.variables_par[f'{var}']['x_axis'], fontsize=26)
                ax.set_ylabel(a_v.variables_par[f'{var}']['y_axis'], fontsize=26)
                ax.set_xlim(a_v.variables_par[f'{var}']['range_low'], 
                            a_v.variables_par[f'{var}']['range_high'])
                ax.set_xlim(a_v.variables_par[f'{var}']['range_low'], 
                            a_v.variables_par[f'{var}']['range_high'])
                if log_scale == 1:
                    ax.set_yscale('log')
                    ax.set_ylim(0.1,1000)
                    ax.set_xscale('log')
                    ax.set_xlim(0,1)
                current_ylim = ax.get_ylim()
                y_range = current_ylim[1]
                y_low = current_ylim[0]
                new_ylim = (0, 1.25 * y_range)
                ax.set_ylim(new_ylim)
                #ax.set_ylim(0,None)
                #if mode == 'pi':
                #    ax.set_ylim(0,np.max(tot_hist)+np.max(tot_hist)*0.4) 
                high = a_v.variables_par[f'{var}']['range_high']
                low = a_v.variables_par[f'{var}']['range_low']
                if log_scale != 1:
                    ax.text(0.01 * high + 0.99 * low, new_ylim[1] * 1.02, r'Belle II preliminary: $B\to\tau\nu$ Had FEI', fontsize = 20, style='italic')
                    ax.text(0.02 * high + 0.99 * low, new_ylim[1] * 0.95, r'$\int \mathcal{L} dt = 362\,fb^{-1}$', fontsize = 20, style='italic')
                    ax.text(0.02 * high + 0.99 * low, new_ylim[1] * 0.90, a_v.latex_decay[mode], fontsize = 20, style='italic')
                else:
                    ax.text(low + low * 0.02, new_ylim[1] + new_ylim[1] * 0.2, r'Belle II preliminary: $B\to\tau\nu$ Had FEI', fontsize = 20, style='italic')
                    ax.text(low + low * 0.5, new_ylim[1] - new_ylim[1] * 0.5, r'$\int \mathcal{L} dt = 362\,fb^{-1}$', fontsize = 20, style='italic')
                    ax.text(high - high * 0.5, new_ylim[1] + new_ylim[1] * 0.2, a_v.latex_decay[mode], fontsize = 20, style='italic')


        A = a_v.variables_par[f'{var}']['range_low']
        B = a_v.variables_par[f'{var}']['range_high']
        fig.tight_layout()
        
        extention = '.pdf'
        if scale_K_BR == 1:
            extention = '_new_K_BR' + extention
        if K_in_event == 1:
            extention = '_K' + extention
        elif Dstar_in_event == 1:
            extention = '_Dstar' + extention
        elif D_in_event == 1:
            extention = '_D' + extention
        elif D_Dstar_in_event == 1:
            extention = '_D_Dstar' + extention
        elif D0_anti_K0isinEvent == 1:
            extention = '_D0K0' + extention
        elif D0KplusisinEvent == 1:
            extention = '_D0Kp' + extention
        elif D0KminusisinEvent == 1:
            extention = '_D0Km' + extention
        elif Dplus_anti_K0isinEvent == 1:
            extention = '_DpK0' + extention
        elif DplusKplusisinEvent == 1:
            extention = '_DpKp' + extention
        elif DplusKminusisinEvent == 1:
            extention = '_DpKm' + extention
        if mbc_sideband == 1:
            extention = '_mbcside' + extention
        if no_data == 1:
            extention = '_nodata' + extention
        if log_scale == 1:
            extention = '_log' + extention
        if multiplicity_corr == 1:
            extention = '_mul' + extention
        if scale_to_offres == 1 and subtract_cont == 0:
            extention = '_scaled_to_offres' + extention
        if scale_to_offres == 1 and subtract_cont == 1:
            extention = 'no_cont' + extention


        plt.savefig(dirName[ROE_var][mode] +f'/fit' + f'/{var}_{mode}_{lumi}_with_cut_{in_cut[mode]}_in_range_{A}_{B}{extention}', bbox_inches='tight')

In [ ]:
import aghast
import boost_histogram as bh
import contextlib

importlib.reload(a_v)

n_sig = {}
n_bkg = {}
n_sig_ = {}
n_bkg_ = {}

n_cont = {}
n_bb = {}
n_cont_ = {}
n_bb_ = {}

epsilon_sel = {}
coeff = {}
pdf = {}

continuum = {}
bkg_list = {}
bb_list = {}
cont_list = {}
sig_list = {}
scale_bkg = {}
tot_list = {}
scale_tot = {}

cont = {}
bbar = {}
bkg = {}
sig = {}
mc_hist_hep = {}
n_bkg = {}
n_sig = {}
scale_bkg = {}
binned_likelihood = {}
m = {}
data_edges = {}
datay = {}
errorp = {}
errorm = {}
total_pdf_x = {}
total_pdf_y = {}
parts = {}
epsilon_sel = {}
toy_br = []
toy_br_errors = []

pdf = {}
toy = {}
pdf_toy = {}
coeff = {}
data = {}
lh = {}

n_tot = 0

BR_pdg = ...
lumi_BBbar = ...
n_BB = ...
BR_Bp_Bm = ...
sigma_BBbar = ...
sigma_BBbar = n_BB * BR_Bp_Bm 

bins_1 = 10
bins_2 = 20

range_1 = (0,1)
range_2 = (-10,26)

zfit.settings.set_seed(seed=1)

for var in ['...','...']:
    bkg_list[var] = {}
    sig_list[var] = {}
    bb_list[var] = {}
    cont_list[var] = {}
    scale_bkg[var] = {}

    for mode in a_v.decay_type:
        scale_bkg[var][mode] = np.concatenate([scale_cont[var][mode],scale_bbar[var][mode]])
        
        continuum[mode] = pd.concat([(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var],(cuts['...'][mode])[var]])
        
        bkg_list[var][mode] = []
        bkg_list[var][mode].append(continuum[mode])
        bkg_list[var][mode].append(cuts['...'][mode][var])
        bkg_list[var][mode].append(cuts['...'][mode][var])
        
        bb_list[var][mode] = []
        bb_list[var][mode].append(cuts['...'][mode][var])
        bb_list[var][mode].append(cuts['...'][mode][var])

        bkg_list[var][mode] = pd.concat(bkg_list[var][mode])
        sig_list[var][mode] = cuts['...'][mode][var]
        cont_list[var][mode] = continuum[mode]
        bb_list[var][mode] = pd.concat(bb_list[var][mode])
        

for mode in a_v.decay_type:
    
    bkg_hist = hist_lib.Hist(hist_lib.axis.Regular(bins_1, 0, 1, name="x",label='Eextra'),
                             hist_lib.axis.Regular(bins_2, -10, 26, name="y",label='MissM2'))
    bkg_hist.fill(x=bkg_list['...'][mode],
                  y=bkg_list['...'][mode],
                  weight=scale_bkg['...'][mode])
    
    sig_hist = hist_lib.Hist(hist_lib.axis.Regular(bins_1, 0, 1, name="x",label='Eextra'),
                             hist_lib.axis.Regular(bins_2, -10, 26, name="y",label='MissM2'))
    sig_hist.fill(x=sig_list['...'][mode],
                  y=sig_list['...'][mode],
                  weight=scale_sig['...'][mode])
    
    n_bkg[mode] = bkg_hist.sum()
    n_sig[mode] = sig_hist.sum()
    
    epsilon_sel[mode] = n_sig[mode] / (a_v.lumi['data']*a_v.sigma_sig)
    coeff[mode] = 2 * sigma_BBbar * epsilon_sel[mode]
        
    if parameters_created[mode] == 0:
        yield_bkg[mode] = zfit.Parameter(f'N_bkg_{mode}', int(n_bkg[mode]))
        if skip_br != 1:
            br_param = zfit.Parameter('BR', BR_pdg)
            skip_br = 1
        coeff_br[mode] = zfit.Parameter(f'coeff_{mode}', coeff[mode],floating = False)
        def dependent_func(mu1, mu2):
            return mu1 * mu2  # or any kind of computation
        yield_sig[mode] = zfit.ComposedParameter(f'N_sig_{mode}', dependent_func, params=[br_param, coeff_br[mode]])
        parameters_created[mode] = 1
        

    br_param.set_value(BR_pdg)
    if yield_bkg[mode].to_dict()['name'] == 'N_bkg_e':
        yield_bkg[mode].set_value(int(n_bkg['e']))
    if yield_bkg[mode].to_dict()['name'] == 'N_bkg_mu':
        yield_bkg[mode].set_value(int(n_bkg['mu']))
    if yield_bkg[mode].to_dict()['name'] == 'N_bkg_pi':
        yield_bkg[mode].set_value(int(n_bkg['pi']))
    if yield_bkg[mode].to_dict()['name'] == 'N_bkg_rho':
        yield_bkg[mode].set_value(int(n_bkg['rho']))
        
    ep_bkg_hist = zfit.pdf.HistogramPDF(bkg_hist,extended=yield_bkg[mode])
    ep_sig_hist = zfit.pdf.HistogramPDF(sig_hist,extended=yield_sig[mode])
        
    pdf[mode] = zfit.pdf.BinnedSumPDF([ep_bkg_hist,ep_sig_hist])
    
    toy[mode] = pdf[mode].create_sampler(n=int(n_bkg[mode]+n_sig[mode]))#, fixed_params=False)
    lh[mode] = zfit.loss.ExtendedBinnedNLL(pdf[mode], toy[mode])

simlh = lh['e'] + lh['mu'] + lh['pi'] + lh['rho']
    
from zfit.minimize import DefaultToyStrategy 

minimizer = zfit.minimize.Minuit(strategy=DefaultToyStrategy())
    
fit_results = []
ntoys = 100

params = simlh.get_params()

print(params)

for i in tqdm(range(ntoys)):  
        
    for mode in a_v.decay_type:
        br_param.set_value(BR_pdg)
        if yield_bkg[mode].to_dict()['name'] == 'N_bkg_e':
            yield_bkg[mode].set_value(int(n_bkg['e']))
        if yield_bkg[mode].to_dict()['name'] == 'N_bkg_mu':
            yield_bkg[mode].set_value(int(n_bkg['mu']))
        if yield_bkg[mode].to_dict()['name'] == 'N_bkg_pi':
            yield_bkg[mode].set_value(int(n_bkg['pi']))
        if yield_bkg[mode].to_dict()['name'] == 'N_bkg_rho':
            yield_bkg[mode].set_value(int(n_bkg['rho']))

    # Generate toys
    for mode in a_v.decay_type:
        n_ = np.random.poisson(int(n_bkg[mode])+int(n_sig[mode]))
        toy[mode].resample(n=n_)
        
    result = minimizer.minimize(simlh)
    

    if result.converged:
        # Calculate uncertainties
        result.hesse()
        fit_results.append(result)
        br_value = result._params[param_br]['value']
        br_error = result._params[param_br]['hesse']['error']
    else:
        br_value = 0
        br_error = 0
        
    #return br_value, br_error

for fit_result in fit_results:
    toy_br.append(fit_result._params[param_br]['value'])
    toy_br_errors.append(fit_result._params[param_br]['hesse']['error'])

In [ ]:
from hepstats.hypotests.parameters import POI
from hepstats.hypotests.calculators import AsymptoticCalculator
from hepstats.hypotests import Discovery

# the null hypothesis
br_poi = POI(param_br, 0)

# construction of the calculator instance
calculator = AsymptoticCalculator(input=simlh, minimizer=minimizer)
calculator.bestfit = result

# equivalent to above
calculator = AsymptoticCalculator(input=result, minimizer=minimizer)

discovery = Discovery(calculator=calculator, poinull=br_poi)
discovery.result()

In [ ]:
from matplotlib.offsetbox import AnchoredText
import scipy.stats as stats

lumi = int(a_v.lumi['data'])

fig, ax = plt.subplots()

toy_hist, toy_bins = np.histogram(toy_br, 
                                  range=(np.min(toy_br),np.max(toy_br)),
                                  bins=50,
                                 )

toy_bin_center = (toy_bins[1:]+toy_bins[:-1])/2

ax.errorbar(x=toy_bin_center,
             y=toy_hist,
             yerr=np.sqrt(toy_hist),
             fmt='ko')
ax.set_title('SimFit ToyMC BR')
ax.set_xlabel('BR', fontsize=26)
ax.set_ylabel('N toy MC', fontsize=26)
ax.set_ylim(0,None)
ax.ticklabel_format(axis='x', style='sci', scilimits=(-4,-4), useMathText=True)

a = AnchoredText(f"mean BR = {round(np.mean(toy_br),7)}", loc=1, pad=0.4, borderpad=0.5)
fig.gca().add_artist(a)

fig.tight_layout()
plt.savefig(dirName[var][mode] +f'/fit/BR_toy_sim_{lumi}fb-1.pdf', bbox_inches='tight')

################

fig, ax = plt.subplots()

toy_hist, toy_bins = np.histogram(toy_br_errors, 
                                  range=(np.min(toy_br_errors),np.max(toy_br_errors)),
                                  bins=50,
                                 )

toy_bin_center = (toy_bins[1:]+toy_bins[:-1])/2

ax.errorbar(x=toy_bin_center,
             y=toy_hist,
             yerr=np.sqrt(toy_hist),
             fmt='ko')
ax.set_title('SimFit ToyMC BR error')
ax.set_xlabel('BR error', fontsize=26)
ax.set_ylabel('N toy MC', fontsize=26)
ax.set_ylim(0,None)
ax.ticklabel_format(axis='x', style='sci', scilimits=(-4,-4), useMathText=True)

a = AnchoredText(f"mean BR error = {round(np.mean(toy_br_errors),7)}", loc=1, pad=0.4, borderpad=0.5)
fig.gca().add_artist(a)

fig.tight_layout()
plt.savefig(dirName[var][mode] +f'/fit/BR_error_toy_sim_{lumi}fb-1.pdf', bbox_inches='tight')

################

fig, ax = plt.subplots()

stack = np.column_stack((toy_br,toy_br_errors))

toy_pulls = []
for i,j in stack:
    toy_pulls.append((i-BR_pdg)/j)

toy_hist, toy_bins = np.histogram(toy_pulls, 
                                  range=(np.min(toy_pulls),np.max(toy_pulls)),
                                  bins=50,
                                 )

toy_bin_center = (toy_bins[1:]+toy_bins[:-1])/2

ax.errorbar(x=toy_bin_center,
             y=toy_hist,
             yerr=np.sqrt(toy_hist),
             fmt='ko')
plt.grid(False)

ax.set_title('SimFit ToyMC BR pulls')
ax.set_xlabel('BR pulls', fontsize=26)
ax.set_ylabel('N toy MC', fontsize=26)
ax.set_xlim(-1,1)
ax.set_ylim(0,None)

fig.tight_layout()
plt.savefig(dirName[var][mode] +f'/fit/BR_pulls_toy_sim_{lumi}fb-1.pdf', bbox_inches='tight')